# Fn to FC
> python module to convert a given Fn into FC automatically

In [ ]:
#| default_exp fn_to_fc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# Importing openai and our custom functions
import openai
import json
import ast
import inspect

from typing import Optional, Union, Callable, Literal
from types import NoneType
from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v

## Example functions

For our first MVP, response generation mostly concern with GPT models answering generic questions and using a single tool for capturing and extracting information from a Youtube Livestream.

In [ ]:
import glob
from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v

In [ ]:
#| export
def capture_youtube_live_frame_and_save(
        link: Optional[str] = None,  # YouTube Live link
    ) -> str:  # Path to the saved image
    """Capture a jpeg file from YouTube Live and save in data directory"""
    if link is not None:
        return str(capture_youtube_live_frame(link))
    return str(capture_youtube_live_frame())

In [ ]:
show_doc(capture_youtube_live_frame_and_save)

---

### capture_youtube_live_frame_and_save

>      capture_youtube_live_frame_and_save (link:Optional[str]=None)

*Capture a jpeg file from YouTube Live and save in data directory*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| link | Optional | None | YouTube Live link |
| **Returns** | **str** |  | **Path to the saved image** |

In [ ]:
#| export
def ask_gpt4v_about_image_file(
        path:str  # Path to the image file
    ) -> str:  # JSON string with quantitative information
    """Tell all about quantitative information from a given image file"""
    info = ask_gpt4v(path)
    return json.dumps(info)

In [ ]:
show_doc(ask_gpt4v_about_image_file)

---

### ask_gpt4v_about_image_file

>      ask_gpt4v_about_image_file (path:str)

*Tell all about quantitative information from a given image file*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| path | str | Path to the image file |
| **Returns** | **str** | **JSON string with quantitative information** |

## Utilities for GPT Function calling

We can use dynamic utilities functions to integrate this to GPT Function calling:  

- Parmater descriptions: extract parameter descriptions from a funcion
- Parameter converter: convert Python parameter types into schema accepted formats
- Schema generator: extract function information into tool schema to bet set for GPT
- Function execution: execute function dynamically based on function names and input arguments

### 1. Extractor for parameter descriptions

In [ ]:
#| export
# Extract parameter comments from the function
def extract_parameter_comments(
        func: Callable  # Function to extract comments from
    ) -> dict[str, str]:  # Dictionary with parameter comments
    """Extract comments for function arguments"""
    # Get the source code of the function
    source = inspect.getsource(func)
    # Parse the source code into an AST
    tree = ast.parse(source)
    
    # Extract comments for function arguments
    comments = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.name == func.__name__:
            # Get arguments and comments in the function
            for arg in node.args.args:
                arg_name = arg.arg
                # Check if there's an inline comment associated with the argument
                if arg.end_lineno and arg.col_offset:
                    # Loop through the source code lines to find the comment
                    lines = source.splitlines()
                    for line in lines:
                        if line.strip().startswith(f"{arg_name}:") and "#" in line:
                            comment = line.split("#")[1].strip()
                            comments[arg_name] = comment
    return comments

In [ ]:
show_doc(extract_parameter_comments)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L36){target="_blank" style="float:right; font-size:smaller"}

### extract_parameter_comments

>      extract_parameter_comments (func:Callable)

*Extract comments for function arguments*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| func | Callable | Function to extract comments from |
| **Returns** | **dict** | **Dictionary with parameter comments** |

Test usage with example functions:

In [ ]:
print(f"Parameters of capture Youtube Live frame function: \
{extract_parameter_comments(capture_youtube_live_frame_and_save)}")
print(f"Parameters of ask GPT4V about image file function: \
{extract_parameter_comments(ask_gpt4v_about_image_file)}")

Parameters of capture Youtube Live frame function: {'link': 'YouTube Live link'}
Parameters of ask GPT4V about image file function: {'path': 'Path to the image file'}


In [ ]:
#| hide
# Test the functions
assert extract_parameter_comments(capture_youtube_live_frame_and_save) == {'link': 'YouTube Live link'}
assert extract_parameter_comments(ask_gpt4v_about_image_file) == {'path': 'Path to the image file'}

### 2. Converter for Python parameter types to acceptable tool schema

In [ ]:
#| export
def param_converter(
        param_type,  # The type of the parameter
        description  # The description of the parameter
    ) -> dict:  # The converted parameter
    """Convert Python parameter types to acceptable types for tool schema"""
    simple_types = {
        str: "string",
        int: "number",
        float: "number",
        bool: "boolean",
    }
    if param_type in simple_types:
        return { "type": simple_types[param_type], "description": description }
    elif param_type == NoneType:
        return { "type": "null", "description": "A default value will be automatically used." }
    
    if hasattr(param_type, '__origin__') and param_type.__origin__ == Union:
        # Recursively convert the types
        descriptions = description.split(" or ")
        subtypes = param_type.__args__
        if len(subtypes) > len(descriptions):
            descriptions = descriptions + ["A description is not provided"] * (len(subtypes) - len(descriptions))

        return {
            "anyOf": [param_converter(subtype, desc) for subtype, desc in zip(subtypes, descriptions)]
        }
    return { "type": "string", "description": description }

Test usage with a more complicated data type `Optional[str]`:

In [ ]:
param_schema = param_converter(Optional[str], "YouTube Live link")
print(json.dumps(param_schema, indent=2))

{
  "anyOf": [
    {
      "type": "string",
      "description": "YouTube Live link"
    },
    {
      "type": "null",
      "description": "A default value will be automatically used."
    }
  ]
}


In [ ]:
#| hide
# Test the function
assert param_schema == { "anyOf": [
        {
            "type": "string",
            "description": "YouTube Live link"
        },
        {
            "type": "null",
            "description": "A default value will be automatically used."
        }
    ]
}

### 3. Tool schema

In [ ]:
#| export
def tool_schema(
        func: Callable  # The function to generate the schema for
    ) -> dict:  # The generated tool schema
    """Automatically generate a schema from its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    param_comments = extract_parameter_comments(func)
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = param_converter(
            param_type, 
            param_comments.get(param_name, "No description provided.")
        )
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = {
        "type": "function",
        "function": {
            "name": func_name,
            "description": func_description,
            "parameters": parameters_schema,
        }
    }
    
    return tool_schema

In [ ]:
show_doc(tool_schema)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/fn_to_fc.py#L15){target="_blank" style="float:right; font-size:smaller"}

### tool_schema

>      tool_schema (func:Callable)

*Automatically generate a schema from its parameters and docstring*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| func | Callable | The function to generate the schema for |
| **Returns** | **dict** | **The generated tool schema** |

In [ ]:
#| export
# Environmental setting up
tools = [tool_schema(fn) for fn in (capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file)]
initial_messages = [{"role":"system", "content":"You are a helpful system administrator. Use the supplied tools to assist the user."}]

In [ ]:
#| echo: false
print(json.dumps(tools, indent=2))

[
  {
    "type": "function",
    "function": {
      "name": "capture_youtube_live_frame_and_save",
      "description": "Capture a jpeg file from YouTube Live and save in data directory",
      "parameters": {
        "type": "object",
        "properties": {
          "link": {
            "anyOf": [
              {
                "type": "string",
                "description": "YouTube Live link"
              },
              {
                "type": "null",
                "description": "A default value will be automatically used."
              }
            ]
          }
        },
        "required": []
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "ask_gpt4v_about_image_file",
      "description": "Tell all about quantitative information from a given image file",
      "parameters": {
        "type": "object",
        "properties": {
          "path": {
            "type": "string",
            "description": "Path to the image file"
     

In [ ]:
#| hide
# Test the function
# Check the schema of the first tool
assert tools[0]["function"]["name"] == "capture_youtube_live_frame_and_save"
assert tools[0]["function"]["description"] == "Capture a jpeg file from YouTube Live and save in data directory"
assert tools[0]["function"]["parameters"]["properties"]["link"] == {
    "anyOf": [
        {
            "type": "string",
            "description": "YouTube Live link"
        },
        {
            "type": "null",
            "description": "A default value will be automatically used."
        }
    ]
}

# Check the schema of the second tool
assert tools[1]["function"]["name"] == "ask_gpt4v_about_image_file"
assert tools[1]["function"]["description"] == "Tell all about quantitative information from a given image file"
assert tools[1]["function"]["parameters"]["properties"]["path"] == {
    "type": "string",
    "description": "Path to the image file"
}
assert tools[1]["function"]["parameters"]["required"] == ["path"]

### 4. Excecution functions

In [ ]:
#| export
# Support functions to handle tool response,where res == response.choices[0].message
def fn_name(res): return res.tool_calls[0].function.name
def fn_args(res): return json.loads(res.tool_calls[0].function.arguments)    
def fn_exec(res): return globals().get(fn_name(res))(**fn_args(res))
def fn_result_content(res):
    """Create a content containing the result of the function call"""
    content = dict()
    content.update(fn_args(res))
    content.update({fn_name(res): fn_exec(res)})
    return json.dumps(content)

In [ ]:
#| export
def generate_messages(
    message: str,  # New message frorm the user
    history : list[dict] = []  # Previous messages
) -> list[dict]:  # List of messages
    """Generate messages from the user and the system"""
    # Copy the history to avoid modifying the original list
    messages = history.copy()
    if len(messages) == 0:
        # Add initial system message if no history
        messages.append({
            "role":"system", 
            "content":"You are a helpful system administrator. Use the supplied tools to assist the user."
        })

    def complete(
            role: Literal["system", "user", "tool", "assistant"],  # The role of the message sender
            content: str,  # The content of the message
            tool_call_id=None):
        """Send completion request with messages, and save the response in messages again"""
        messages.append({"role":role, "content":content, "tool_call_id":tool_call_id})
        response = openai.chat.completions.create(
            model="gpt-4o", 
            messages=messages, 
            tools=tools
        )
        res = response.choices[0].message
        messages.append(res.to_dict())
        if res.to_dict().get('tool_calls'):
            complete(role="tool", content=fn_result_content(res), tool_call_id=res.tool_calls[0].id)
        return messages[-1]['role'], messages[-1]['content']
    
    complete("user", message)
    return messages

In [ ]:
show_doc(generate_messages)

---

### generate_messages

>      generate_messages (message:str, history:list[dict]=[])

*Generate messages from the user and the system*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| message | str |  | New message frorm the user |
| history | list | [] | Previous messages |
| **Returns** | **list** |  | **List of messages** |

Test usage with a conversation:

In [ ]:
#| eval: false
messages = generate_messages("Hi, can you capture YouTube Live and retrieve information from it? Use the default Youtube link.")
for message in messages:
    print(f"{message['role'].capitalize()}: {message['content']}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
System: You are a helpful system administrator. Use the supplied tools to assist the user.
User: Hi, can you capture YouTube Live and retrieve information from it? Use the default Youtube link.
Assistant: None
Tool: {"link": null, "capture_youtube_live_frame_and_save": "../data/fail_2024.11.04_18:00:07_nowhere.jpg"}
Assistant: None
Tool: {"path": "../data/fail_2024.11.04_18:00:07_nowhere.jpg", "ask_gpt4v_about_image_file": "{\"timestamp\": \"2024-11-04T17:52:45\", \"dimensions\": {\"width\": 1280, \"height\": 720}, \"buildings\": {\"number_of_buildings\": 0}, \"vehicles\": {\"number_of_vehicles\": 0}, \"waterbodies\": {\"visible\": false}, \"street_lights\": {\"nu

Let's try to continue this conversation:

In [ ]:
#| eval: false
new_messages = generate_messages("Can you explain this timestamp", messages)
for message in new_messages:
    print(f"{message['role'].capitalize()}: {message['content']}")

System: You are a helpful system administrator. Use the supplied tools to assist the user.
User: Hi, can you capture YouTube Live and retrieve information from it? Use the default Youtube link.
Assistant: None
Tool: {"link": null, "capture_youtube_live_frame_and_save": "../data/fail_2024.11.04_18:00:07_nowhere.jpg"}
Assistant: None
Tool: {"path": "../data/fail_2024.11.04_18:00:07_nowhere.jpg", "ask_gpt4v_about_image_file": "{\"timestamp\": \"2024-11-04T17:52:45\", \"dimensions\": {\"width\": 1280, \"height\": 720}, \"buildings\": {\"number_of_buildings\": 0}, \"vehicles\": {\"number_of_vehicles\": 0}, \"waterbodies\": {\"visible\": false}, \"street_lights\": {\"number_of_street_lights\": 5}, \"people\": {\"approximate_number\": 0}, \"lighting\": {\"time_of_day\": \"night\", \"artificial_lighting\": \"prominent\"}, \"visibility\": {\"clear\": false}, \"sky\": {\"visible\": true, \"light_conditions\": \"dark\"}}"}
Assistant: I captured a frame from the YouTube Live stream using the defau

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()